# 4.1. Markov-Entscheidungsprozesse

>## <ins>Table of contents</ins>
>* [**4.1.1. Modellierung einer Umgebung**](#4_1_1)
>    * [**Das Markov-Entscheidungsprozess**](#4_1_1_a)
>    * [**Strategie**](#4_1_1_b)
>* [**4.1.2. Iterative Entwicklung der Zustandsnutzen**](#4_1_2)
>* [**4.1.3. Iterative Strategieentwicklung**](#4_1_3)
>
>## <ins>Beispiele</ins>
>* [**Beispiel 1**: Staubsauger](#b1)
>* [**Beispiel 2**: Episoden $\pi$ des Beispiels *Staubsauger*](#b2)
>* [**Beispiel 3**: Nutzen einer Episode $e$ *Staubsauger*](#b3)
>* [**Beispiel 4**: Nutzen einer Strategie $\pi$ von *Staubsauger*](#b4)


## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Damit ein Agent lernt Entscheidungen in einer gewissen Umgebung zu treffen, muss diese Umgebung erst modelliert werden.

## 4.1.1. Modellierung einer Umgebung <a name="4_1_1"></a>

### Der Markov-Entscheidungsprozess <a name="4_1_1_a"></a>
Ein MDP ist ein mathematisches Modell, das die Umgebung eines lernenden Agenten beschreibt. Es besteht aus einem <ins>Zustandsraum</ins>, einem <ins>Aktionsraum</ins>, einer <ins>Übergangswahrscheinlichkeitsfunktion</ins>, einer <ins>Belohnungsfunktion</ins>, einem <ins>Startzustand</ins> und einer <ins>Menge von Endzuständen</ins>.


> **Definition 1.: Markov-Entscheidungsprozess**
>
> Ein Markov-Entscheidungsprozess (engl. Markov decision process, MDP) D ist ein Tupel $D = (S,A,P,R,s_0,S_t)$ mit folgenden Eigenschaften:
> 1. $S$ ist eine Menge von Zuständen (der Zustandsraum)
> 2. $A$ ist eine Menge von Aktionen (der Aktionsraum)
> 3. $P : (S \setminus S_t) \times A \times S \rightarrow [0,1]$ ist eine Funktion (die Übergangswahrscheinlichkeitsfunktion) mit $\sum_{s' \in S} P(s,a,s') = 1$ für alle $s \in S$, $a \in A$.
> 4. $R : (S \setminus S_t) \times A \times S \rightarrow \mathbb{R}$ ist eine beliebige reellwertige Funktion (die Belohnungsfunktion, engl. reward function).
> 5. $s_0 \in S$ ist der Startzustand.
> 6. $S_t \subseteq S$ ist die Menge der Zielzustände.



In jedem *Zustand* $s \in S$ kann der Agent eine *Aktion* $a \in A$ ausführen, die zu einem neuen Zustand führt. Die *Übergangswahrscheinlichkeitsfunktion* $P$ gibt die Wahrscheinlichkeit an, dass eine bestimmte Aktion in einem bestimmten Zustand zu einem bestimmten neuen Zustand führt. Die *Belohnungsfunktion* $R$ gibt an, welche Belohnung (oder Strafe, wenn negativ) der Agent erhält, wenn er eine bestimmte Aktion in einem bestimmten Zustand ausführt und zu einem bestimmten neuen Zustand wechselt. 

Das **Ziel** des Agenten im Reinforcement Learning ist es, eine Strategie zu lernen, die die kumulative Belohnung über die Zeit <ins>maximiert</ins>. Dies wird oft durch eine Kombination aus **Exploration** (neue Aktionen ausprobieren, um mehr über die Umgebung zu erfahren) und **Exploitation** (die besten bekannten Aktionen ausführen) erreicht. 

Es ist wichtig zu beachten, dass MDPs die Markov-Eigenschaft haben, d.h., die Wahrscheinlichkeit, einen bestimmten neuen Zustand zu erreichen, hängt nur vom aktuellen Zustand und der ausgeführten Aktion ab, nicht von der vorherigen Geschichte des Agenten. Dies vereinfacht das Problem erheblich, da der Agent nicht die gesamte Geschichte seiner Interaktionen mit der Umgebung speichern muss. 

>
> #### **Beipiel:** Staupsauger
>
> 1. **Zusatndsmenge $S$**: Jeder Zustand könnte eine Kombination aus der aktuellen Position des Roboters (z.B. die verschiedenen Räume der Wohnung) und dem Ladestand der Batterie sein.
>    Wenn es beispielsweise vier Räume gibt, in denen der Roboter sich aufhalten kann, und die Batterie drei verschiedene Ladestufen hat (niedrig, mittel, hoch), dann gibt es insgesamt 12 mögliche Zustände.
> 
> 2. **Aktionsmenge $A$**: ”saugen“, ”laden“ oder sich in einen Nachbarraum bewegen.
>    In allgemeinen MDPs ist das Resultat einer Aktion nicht immer deterministisch bestimmt (beispielsweise kann das Laden des Roboters fehlschlagen, wenn die Batterie eine Fehlfunktion hat).
>    
> 3. **Übergangswahrscheinlichkeitsfunktion $P$**:  Diese Funktion könnte davon abhängen, wie der Roboter sich bewegt und wie die Batterie entladen wird. Zum Beispiel könnte die Wahrscheinlichkeit, dass der Roboter sich erfolgreich zu einem anderen Raum bewegt, von der aktuellen Ladung der Batterie abhängen. 
>
> 4. **Belohnungsfunktion $R$**: Die Belohnung könnte positiv sein, wenn der Roboter einen Raum saugt, und negativ sein, wenn der Roboter versucht, sich zu bewegen, aber die Batterie leer ist. Die genaue Definition der Belohnungsfunktion hängt davon ab, was wir den Roboter optimieren wollen. Wenn wir wollen, dass der Roboter so viel wie möglich saugt, könnten wir eine hohe positive Belohnung für das Saugen geben. Wenn wir wollen, dass der Roboter seine Batterie effizient nutzt, könnten wir eine negative Belohnung für das Bewegen geben, wenn die Batterie fast leer ist.


\})$$

#### **Beispiel 1.** <a name="b1"></a> Ein stark vereinfachtes Modell des Staubsaugerroboterproblems

Wir gehen davon aus, dass unsere Wohnung aus zwei Räumen, r1 und r2, besteht, die entweder sauber oder verschmutzt sein können. Der Zustandsraum ist dann definiert durch:

$$S^{vc} = \{s^{1,1}_1, s^{1,1}_2, s^{0,1}_1, s^{0,1}_2, s^{1,0}_1, s^{1,0}_2, s^{0,0}_1, s^{0,0}_2, s^t\}$$

wobei der Zustand $s^{k,l}_i$ repräsentiert, dass der Roboter in Raum $r_i$ ist:
- Raum $r_1$ bei $k = 0$ sauber und bei $k = 1$ verschmutzt ist
- und Raum $r_2$ bei $l = 0$ sauber und bei $l = 1$ verschmutzt ist.

| Zustand | Raum $r_i$ | Raum $r_1$ | Raum $r_2$ |
|---------|------------|------------|------------|
| $s^{0,0}_1$ | Roboter in Raum $r_1$ | Sauber | Sauber |
| $s^{1,0}_1$ | Roboter in Raum $r_1$ | Verschmutzt | Sauber |
| $s^{0,1}_1$ | Roboter in Raum $r_1$ | Sauber | Verschmutzt |
| $s^{1,1}_1$ | Roboter in Raum $r_1$ | Verschmutzt | Verschmutzt |
| $s^{0,0}_2$ | Roboter in Raum $r_2$ | Sauber | Sauber |
| $s^{1,0}_2$ | Roboter in Raum $r_2$ | Verschmutzt | Sauber |
| $s^{0,1}_2$ | Roboter in Raum $r_2$ | Sauber | Verschmutzt |
| $s^{1,1}_2$ | Roboter in Raum $r_2$ | Verschmutzt | Verschmutzt |


$s^t$ ist der Zielzustand und $s^{1,1}_1$ ist der Startzustand. 

In jedem Zustand (außer $s^t$) hat der Roboter drei mögliche Aktionen:

1. move: Gehe in den anderen Raum.
2. clean: Reinige den aktuellen Raum.
3. charge: Lade die Batterie.

Also, $A_{vc} = \{move, clean, charge\}$. 

Die Übergangswahrscheinlichkeiten (modelliert durch $P_{vc}$) sind so definiert, dass sich in Raum $r_1$ eine Ladestation befindet und nach Ausführung der Aktion charge in $r_1$ der aktuelle "Tag" endet und wir zum finalen Zustand $s^t$ wechseln. 

Die Aktion move ist zu 90% erfolgreich (d.h., mit Wahrscheinlichkeit 0.1 bleibt der Roboter im aktuellen Raum) und die Aktion clean ist zu 80% erfolgreich (d.h., ein schmutziger Raum bleibt mit Wahrscheinlichkeit 0.2 schmutzig; ein sauberer Raum bleibt sauber). 

Die Belohnungen ($R_{vc}$) sind wie folgt definiert:

1. Wenn der Roboter erfolgreich einen zuvor schmutzigen Raum reinigt, erhält er 10 Punkte.
2. Wenn der Roboter einen sauberen Raum reinigt, erhält er -2 Punkte (da er unnötig Energie verbraucht hat).
3. Wenn der Roboter in Raum $r_2$ zu laden versucht, erhält er -5 Punkte (er hat bei dem Versuch seine Batterie beschädigt).
4. Wenn der Roboter in $r_1$ auflädt, ohne vorher alle Räume gereinigt zu haben, erhält er -7 Punkte.
5. Für jede Bewegung mittels move (unabhängig davon, ob erfolgreich oder nicht) erhält der Roboter -1 Punkt (der Roboter soll möglichst effizient die Wohnung reinigen).

Für alle weiteren Situationen beträgt die Belohnung 0 Punkte.

Wir erhalten: $$ D_{vc} = (S_{vc}, A_{vc}, P_{vc}, R_{vc}, s^{1,1}_1, S_{vc}^t = \{s^t\})$$

![markov_staubsauger_beispiel](./markov_staubsauger-beispiel.PNG)


### Die konstante und optimale Strategie <a name="4_1_1_b"></a>

Um den Agenten in einem MDP zu steuern, besitzt dieser eine *Strategie* (engl. policy), die (im einfachsten Fall) für jeden Zustand die auszuwählende Aktion bestimmt.

>**Definition 2.:** Strategie
>
>Sei $D = (S,A,P,R,s_0,S_t)$ ein MDP. Eine konstante Strategie $\pi$ für $D$ ist eine Funktion $$\pi : S \setminus S_t \rightarrow A$$


Eine **konstante Strategie** ist eine Funktion, die jedem Zustand eine bestimmte Aktion zuweist. Sie ist deterministisch, d.h., sie gibt für jeden Zustand genau eine Aktion vor.

Es gibt jedoch auch **probabilistische Strategien**, die jedem Zustand eine Wahrscheinlichkeitsverteilung über die möglichen Aktionen zuweisen. Diese Art von Strategie kann nützlich sein, wenn es Unsicherheit oder Zufälligkeit in der Umgebung gibt. 

In vielen Fällen, insbesondere bei komplexen MDPs, kann es vorteilhaft sein, probabilistische Strategien zu verwenden, um eine bessere Leistung zu erzielen. Aber für den Anfang ist es absolut sinnvoll, sich auf konstante Strategien zu konzentrieren.

#### **Beispiel 2** <a name="b2"></a> Fortsetzung von Beispiel 1

Eine mögliche Strategie $π_{vc}$ ist wie folgt definiert:

$$\pi_{vc}(s^{1,1}_{1}) = \pi_{vc}(s^{1,0}_{1}) = \pi_{vc}(s^{1,1}_{2}) = \pi_{vc}(s^{0,1}_{2}) = \text{clean}$$
$$\pi_{vc}(s^{0,1}_{1}) = \pi_{vc}(s^{1,0}_{2}) = \pi_{vc}(s^{0,0}_{2}) = \text{move}$$
$$\pi_{vc}(s^{0,0}_{1}) = \text{charge}$$

Das bedeutet, wenn der Roboter sich in einem verschmutzten Raum befindet, führt er die Aktion `clean` aus. Wenn der aktuelle Raum sauber ist und der andere Raum verschmutzt ist, wechselt der Roboter in den anderen Raum (`move`). Wenn beide Räume sauber sind, wechselt der Roboter zuerst in Raum `r1` und lädt dann auf (`charge`).

Die Aufgabe eines Offline-Lernverfahrens besteht darin, <ins>eine optimale Strategie `π*` für ein gegebenes festes MDP zu erlernen</ins>. 

Eine **optimale Strategie** ist diejenige, die die akkumulierte erwartete Belohnung maximiert, die der Agent durch die Anwendung der Strategie erhält. 

Um dies zu definieren, benötigen wir einige zusätzliche Konzepte.

>**Definition 3.:** Episode
>
>Sei $D = (S,A,P,R,s_0,S_t)$ ein MDP. Eine Episode `e` in $D$ ist eine (potenziell unendliche) Folge $e = (s_0, a_1, s_1, a_2, s_2, ...)$ mit $s_0, s_1 ... \in S$ und $a_0, a_1 ... \in A$ Die Wahrscheinlichkeit $P(e)$ ist definiert durch
>$$P(e) = \prod_{i>0} P(s_{i-1},a_i,s_i)$$

Eine **Episode** `e` in einem MDP ist eine (potenziell unendliche) Folge von Zuständen und Aktionen, die mit dem Anfangszustand `s0` beginnt und die Aktionen `a1, a2, ...` ausführt, wobei der Agent in den Zuständen `s1, s2, ...` landet. 

Eine Episode ist **initial**, wenn sie mit dem Anfangszustand $s_0$ beginnt bzw. wenn $s_0 = s^0$ gilt, und sie ist **terminierend**, wenn sie mit einem Zustand $s_n$ endet, der zu den Endzuständen $S^t$ gehört d.h. $s_n \in S^t$ .

> Der unendliche Fall tritt beispielsweise für den Roboter aus unserem Staubsaugerbeispiel auf, wenn dieser permanent nur die move-Aktion ausführt oder wenn eine move-Aktion permanent fehlschlägt (wobei für solch eine Episode die Wahrscheinlichkeit gegen 0 geht).

Jede Episode akkumuliert gewisse Belohnungen und die Summe aller Belohnungen nennen wir **Nutzen** der Episode (engl. utility oder auch return).

>**Definition 4. Nutzen einer Episode γ**
>
>Sei $D = (S,A,P,R,s_0,St)$ ein MDP und $e = (s_0,a_1,s_1,a_2,s_2,...)$ eine Episode in D. Für $\gamma \in [0,1]$ heißt $U^\gamma_D(e)$ definiert via
>$$U^\gamma_D(e) = \sum_{i>0} \gamma^{i-1}R(s_{i-1},a_i,s_i)$$
>der mit $\gamma$ diskontierte Nutzen von $e$ in $D$.

Der **Nutzen** einer Episode ist die Summe aller akkumulierten Belohnungen, wobei jede Belohnung mit einem **Discountfaktor** `γ` gewichtet wird. Dieser Diskontfaktor liegt im Bereich von 0 bis 1 und bestimmt, wie stark zukünftige Belohnungen im Vergleich zu sofortigen Belohnungen bewertet werden. 
- Bei kleinen Werten von `γ` werden Strategien bevorzugt, die schnell hohe Belohnungen erhalten,
- während bei größeren Werten nahe 1 spätere Belohnungen wichtiger werden.

> Üblicherweise wählt man für `γ` einen Wert echt kleiner als 1 (< 1), der aber immer noch recht nahe an 1 ist (beispielsweise 0.9 oder 0.99). Für Werte echt kleiner als 1 ist es auch gewährleistet, dass $U^γ_D(e)$ stets endlich ist, selbst bei unendlichen langen Episoden mit positiver Wahrscheinlichkeit

#### **Beispiel 3** <a name="b3"></a> Fortsetzung von Beispiel 2

Betrachten wir die folgende (initiale und terminierende) Episode $e_1$, definiert durch

$$e_1 = (s_{1,1}^1, \text{clean}, s_{0,1}^1, \text{move}, s_{0,1}^2, \text{clean}, s_{0,0}^2, \text{move}, s_{0,0}^1, \text{charge}, s_t)$$

Der Roboter reinigt also zunächst Raum $r_1$, wechselt dann in Raum $r_2$, reinigt diesen, wechselt wieder in Raum $r_1$ und lädt sich dann auf. Alle Aktionen sind hier also erfolgreich. 

Von Beispiel 1 haben wir bereits einige der Übergangswahrscheinlichkeiten definiert:
- Die Aktion `move` ist zu `90% erfolgreich`, d.h., mit Wahrscheinlichkeit 0.1 bleibt der Roboter im aktuellen Raum.
- Die Aktion `clean` ist zu `80% erfolgreich`, d.h., ein schmutziger Raum bleibt mit Wahrscheinlichkeit 0.2 schmutzig; ein sauberer Raum bleibt sauber.


Die Wahrscheinlichkeit einer Episode $P(e_1)$ kann berechnet werden, indem die Wahrscheinlichkeiten aller Zustandsübergänge in der Episode multipliziert werden. In $e_1$ sind alle Aktionen erfolgreich, daher können wir die Wahrscheinlichkeiten direkt aus den gegebenen Übergangswahrscheinlichkeiten entnehmen.

Die Episode $e_1$ besteht aus den folgenden Zustandsübergängen:

1. $s_{1,1}^1$ (clean) $\rightarrow$ $s_{0,1}^1$: Der Roboter reinigt Raum $r_1$. Die Wahrscheinlichkeit für diesen Übergang ist 0.8 (da die Aktion `clean` zu 80% erfolgreich ist).
2. $s_{0,1}^1$ (move) $\rightarrow$ $s_{0,1}^2$: Der Roboter wechselt in Raum $r_2$. Die Wahrscheinlichkeit für diesen Übergang ist 0.9 (da die Aktion `move` zu 90% erfolgreich ist).
3. $s_{0,1}^2$ (clean) $\rightarrow$ $s_{0,0}^2$: Der Roboter reinigt Raum $r_2$. Die Wahrscheinlichkeit für diesen Übergang ist 0.8.
4. $s_{0,0}^2$ (move) $\rightarrow$ $s_{0,0}^1$: Der Roboter wechselt zurück in Raum $r_1$. Die Wahrscheinlichkeit für diesen Übergang ist 0.9.
5. $s_{0,0}^1$ (charge) $\rightarrow$ $s_t$: Der Roboter lädt sich auf und erreicht den Zielzustand. Da sich die Ladestation in Raum $r_1$ befindet, ist dieser Übergang immer erfolgreich, d.h., die Wahrscheinlichkeit ist 1.


Wir erhalten als Wahrscheinlichkeit $P_{e_1}$ hier

$$P(e_1) = P(s_{1,1}^1, \text{clean}, s_{0,1}^1)P(s_{0,1}^1, \text{move}, s_{0,1}^2)P(s_{0,1}^2, \text{clean}, s_{0,0}^2)P(s_{0,0}^2, \text{move}, s_{0,0}^1)P(s_{0,0}^1, \text{charge}, s_t)$$
$$= 0.8 \cdot 0.9 \cdot 0.8 \cdot 0.9 \cdot 1.0 = 0.5184$$

und mit $\gamma = 0.9$ als Nutzen

$$U^\gamma_D(e_1) = R(s_{1,1}^1, \text{clean}, s_{0,1}^1)+\gamma R(s_{0,1}^1, \text{move}, s_{0,1}^2)+\gamma^2P(s_{0,1}^2, \text{clean}, s_{0,0}^2)+\gamma^3R(s_{0,0}^2, \text{move}, s_{0,0}^1)+\gamma^4R(s_{0,0}^1, \text{charge}, s_t)$$
$$= 10+0.9 \cdot(-1)+0.9^2 \cdot 10+0.9^3 \cdot(-1)+0.9^4 \cdot 0 = 10-0.9+8.1-0.729 = 16.471$$

Schauen wir uns eine weitere Episode $e_2$ mit

$$e_2 = (s_{1,1}^1, \text{clean}, s_{1,1}^1, \text{clean}, s_{0,1}^1, \text{charge}, s_t)$$

an. Hier versucht der Roboter zunächst vergeblich, Raum $r_1$ zu reinigen. Nachdem es beim zweiten Versuch geklappt hat, geht er direkt an die Ladestation. Wir erhalten

$$P(e_2) = P(s_{1,1}^1, \text{clean}, s_{1,1}^1)P(s_{1,1}^1, \text{clean}, s_{0,1}^1)P(s_{0,1}^1, \text{charge}, s_t) = 0.2 \cdot 0.8 \cdot 1 = 0.16$$

und

$$U^\gamma_D(e_2) = R(s_{1,1}^1, \text{clean}, s_{1,1}^1)+\gamma R(s_{1,1}^1, \text{clean}, s_{0,1}^1)+\gamma^2R(s_{0,1}^1, \text{charge}, s_t) = 0+0.9 \cdot 10+0.9^2(-7) = 9-5.67 = 3.33$$




Nun können wir den Nutzen einer Strategie $\pi$ als den erwarteten Nutzen aller durch $\pi$ generierten Episoden definieren:

Eine Episode $e = (s_0,a_1,s_1,a_2,s_2,...)$ wird aus $\pi$ generiert, geschrieben $\pi \sim e$, wenn $\pi(s_{i-1}) = a_i$ für alle $i$ gilt. Das heißt, für jeden Zustand $s_{i-1}$ in der Episode wählt die Strategie π die Aktion $a_i$​ aus, die als nächstes ausgeführt wird. Ist $e$ zusätzlich initial (d.h., sie beginnt mit dem initialen Zustand des Problems), so schreiben wir $\pi \sim_0 e$._0 e$.

Der **Nutzen** einer Strategie $\pi$ ist der erwartete Nutzen aller durch $\pi$ generierten Episoden. Das heißt, wir betrachten alle möglichen Episoden, die durch die Strategie $\pi$ generiert werden könnten, berechnen den Nutzen jeder Episode (z.B. durch Summieren der Belohnungen in der Episode), und nehmen dann den Durchschnitt dieser Nut, wobei wir jede Episode mit ihrer Wahrscheinlichkeit gewichten.enwerte. Dies gibt uns eine Maßzahl dafür, wie gut die Strategie $\pi$ im Durchschngen haben.

> **Definition 5. Nutzen von $\pi$ in $D$**
>
> Sei $D = (S,A,P,R,s_0,S_t)$ ein Markov-Entscheidungsprozess (MDP), $\gamma \in [0,1]$ und $\pi : S \setminus S_t \rightarrow A$ eine Strategie für $D$. Der Nutzen $U^\gamma_D(\pi)$ von $\pi$ in $D$ ist definiert durch
> 
$$U^\gamma_D(\pi) = E_{\pi \sim_0 e} \left[ U^\gamma_D(e) \right] = \sum_{\pi \sim_0 e} P(e)U^\gamma_D(e)
 $$

Mit anderen Worten, der Nutzen $U^\gamma_D(\pi)$ von $\pi$ in $D$ ist der durchschnittliche Nutzen aller aus $\pi$ generierten initialen Episoden, gewichtet nach deren Wahrscheinlichkeit.


#### **Beispiel 4.**<a name="b4"></a>  Fortsetzung Beispiel 3

Wir führen Beispiel 3 fort und betrachten die Strategie $\pi_{vc}$ aus Beispiel 2. 

Beachten Sie, dass jede von $\pi_{vc}$ generierte initiale Episode die folgende Struktur $e_{n1,n2,n3,n4}$ für alle $n1,n2,n3,n4 \in \mathbb{N}^+$ hat:

$$
e_{n1,n2,n3,n4} = (\underbrace{s^{1,1}_1, \text{clean},\ldots,s^{1,1}_1, \text{clean}}_{n_1 \text{mal}},\underbrace{s^{0,1}_1, \text{move},\ldots,s^{0,1}_1, \text{move}}_{n_2-\text{mal}},\underbrace{s^{0,1}_2, \text{clean},\ldots,s^{0,1}_2, \text{clean}}_{n_3-\text{mal}},\underbrace{s^{0,0}_2, \text{move},\ldots,s^{0,0}_2, \text{move}}_{n_4-\text{mal}},s^{0,0}_1, \text{charge},s_t)
$$
- Der Roboter beginnt im Zustand $s^{1,1}_1$ und führt die Aktion `clean` $n_1$-mal aus. Dies wird durch den Ausdruck $\underbrace{s^{1,1}_1, \text{clean},\ldots,s^{1,1}_1, \text{clean}}_{n_1 \text{mal}}$ dargestellt.
- Dann führt der Roboter die Aktion `move` $n_2$-mal aus, während er sich im Zustand $s^{0,1}_1$ befindet. Dies wird durch den Ausdruck $\underbrace{s^{0,1}_1, \text{move},\ldots,s^{0,1}_1, \text{move}}_{n_2-\text{mal}}$ dargestellt.
- Als nächstes führt der Roboter die Aktion `clean` $n_3$-mal aus, während er sich im Zustand $s^{0,1}_2$ befindet. Dies wird durch den Ausdruck $\underbrace{s^{0,1}_2, \text{clean},\ldots,s^{0,1}_2, \text{clean}}_{n_3-\text{mal}}$ dargestellt.
- Schließlich führt der Roboter die Aktion `move` $n_4$-mal aus, während er sich im Zustand $s^{0,0}_2$ befindet. Dies wird durch den Ausdruck $\underbrace{s^{0,0}_2, \text{move},\ldots,s^{0,0}_2, \text{move}}_{n_4-\text{mal}}$ dargestellt.
- Der Roboter endet die Episode, indem er sich im Zustand $s^{0,0}_1$ auflädt und dann in den Endzustand $s_t$ übergeht.

Die Zahlen $n_1$, $n_2$, $n_3$ und $n_4$ repräsentieren die Anzahl der Male, die jede Aktion fehlschlägt, bevor sie erfolgreich ist. Daher unterscheiden sich die einzelnen Episoden darin, wie oft das erste `clean`, das erste `move`, das zweite `clean` und das zweite `move` fehlschlagen. 

Die Wahrscheinlichkeit, dass eine Aktion erfolgreich ist, wird mit der Wahrscheinlichkeit eines Fehlschlags multipliziert, um die Gesamtwahrscheinlichkeit für eine Sequenz von Aktionen zu berechnen. In unserem Modell funktioniert der `charge`-Übergang immer, wenn der Roboter sich im Raum r1​ befindet, da dort die Ladestation steht. Daher ist die Wahrscheinlichkeit, dass der charge-Übergang erfolgreich ist, `1`.

Es gilt
$$
P(e_{n1,n2,n3,n4}) = 0.2^{n1-1} \cdot 0.8 \cdot 0.1^{n2-1} \cdot 0.9 \cdot 0.2^{n3-1} \cdot 0.8 \cdot 0.1^{n4-1} \cdot 0.9 \cdot 1.0 $$
$$= 0.5184 \cdot 0.2^{n1+n3-2} \cdot 0.1^{n2+n4-2}
$$



und weiterhin

$$
U^\gamma_D(e_{n1,n2,n3,n4}) = \gamma^{n1-1}10+\gamma^{n1+n2+n3-1}10-\sum_{i=n1}^{n1+n2-1}\gamma^i-\sum_{i=n1+n2+n3}^{n1+n2+n3+n4-1}\gamma^i
$$

>1. $\gamma^{n1-1}10$: Dieser Term repräsentiert die Belohnung von 10 Punkten, die der Roboter erhält, wenn er erfolgreich einen zuvor schmutzigen Raum reinigt. Diese Aktion findet nach $n1-1$ Schritten statt, daher wird die Belohnung mit $\gamma^{n1-1}$ diskontiert.
>
>2. $\gamma^{n1+n2+n3-1}10$: Dieser Term repräsentiert die Belohnung von 10 Punkten, die der Roboter erhält, wenn er erfolgreich einen weiteren zuvor schmutzigen Raum reinigt. Diese Aktion findet nach $n1+n2+n3-1$ Schritten statt, daher wird die Belohnung mit $\gamma^{n1+n2+n3-1}$ diskontiert.
>
>3. $-\sum_{i=n1}^{n1+n2-1}\gamma^i$: Dieser Term repräsentiert die Kosten von -1 Punkt für jede Bewegung mittels `move`, die der Roboter ausführt, um vom Raum $r1$ zum Raum $r2$ zu gelangen. Da diese Aktionen zwischen den Schritten $n1$ und $n1+n2-1$ stattfinden, werden die Kosten entsprechend diskontiert.
>
>4. $-\sum_{i=n1+n2+n3}^{n1+n2+n3+n4-1}\gamma^i$: Dieser Term repräsentiert die Kosten von -1 Punkt für jede Bewegung mittels `move`, die der Roboter ausführt, um vom Raum $r2$ zurück zum Raum $r1$ zu gelangen. Da diese Aktionen zwischen den Schritten $n1+n2+n3$ und $n1+n2+n3+n4-1$ stattfinden, werden die Kosten entsprechend diskontiert.

Die Summe dieser Terme gibt den erwarteten kumulierten diskontierten Ertrag für die Episode $e_{n1,n2,n3,n4}$.

Damit folgt

$$
U^\gamma_D(\pi_{vc}) = \sum_{n1,n2,n3,n4 \in \mathbb{N}^+} P(e_{n1,n2,n3,n4})U^\gamma_D(e_{n1,n2,n3,n4}) $$
$$= \sum_{n1,n2,n3,n4 \in \mathbb{N}^+} \left( 0.5184 \cdot 0.2^{n1+n3-2} \cdot 0.1^{n2+n4-2} \right) \left( \gamma^{n1-1}10+\gamma^{n1+n2+n3-1}10-\sum_{i=n1}^{n1+n2-1}\gamma^i-\sum_{i=n1+n2+n3}^{n1+n2+n3+n4-1}\gamma^i \right)
$$

Mit numerischen Methoden erhalten wir $U^\gamma_D(\pi_{vc}) \approx 15.529$.

Eine Strategie $\pi^*$ ist nun optimal, wenn sie den Nutzen maximiert:

$$
\pi^* = \arg\max_\pi U^\gamma_D(\pi)
$$

Eine naive Methode, eine optimale Strategie zu bestimmen, besteht darin, für alle möglichen Strategien ihren Nutzen zu berechnen und eine Strategie mit maximalem Nutzen auszuwählen. Betrachten wir nur konstante Strategien, so ist deren Anzahl $|A|^{|S \setminus S_t|}$ und dies macht natürlich diese naive Methode nicht praktikabel. Im Folgenden werden wir uns zwei effektivere Verfahren zur Ermittlung einer optimalen Strategie $\pi^*$, für den Fall, dass der MDP $D$ bekannt ist, anschauen.


## 4.1.2.Iterative Entwicklung der Zustandsnutzen <a name="4_1_2"></a>
[policy and value iteration](https://www.youtube.com/watch?v=l87rgLg90HI)

Die iterative Entwicklung des Zustandsnutzens (Value Iteration) ist ein wichtiger Algorithmus in der Verstärkungslerntheorie. Wir haben zuvor schon den Nutzen von Episoden und Strategien definiert. Der Nutzen eines Zustands $s$ bezüglich einer Strategie $\pi$ ist definiert als der erwartete Nutzen aller Episoden, die in $s$ starten.

>**Definition 6:** Nutzen von Zuständen
>
> Sei $D = (S,A,P,R,s_0,S_t)$ ein Markov-Entscheidungsprozess (MDP), $\gamma \in [0,1]$ ist der Diskontierungsfaktor, $s \in S$ ist ein Zustand, und $\pi : S \setminus S_t \rightarrow A$ ist eine Strategie für $D$. Der Nutzen $U^\gamma_D(s | \pi)$ von $s$ bezüglich $\pi$ in $D$ ist definiert durch:
>
>$$U^\gamma_D(s | \pi) = E_{\pi\sim e=(s,a_1,s_1,...)}\left[U^\gamma_D(e)\right] = \sum_{\pi\sim e=(s,a_1,s_1,...)} P(e)U^\gamma_D(e)$$

Diese Formel besagt, dass der Nutzen eines Zustands s unter einer Strategie π gleich dem erwarteten (E) Nutzen aller Episoden e ist, die in s starten und der Strategie π folgen. Dieser Erwartungswert wird berechnet, indem über alle solchen Episoden summiert wird, wobei jede Episode mit ihrer Wahrscheinlichkeit P(e) gewichtet wird und der Nutzen dieser Episode UγD(e) berücksichtigt wird.

Wenn $\pi^*$ optimal ist, schreiben wir statt $U^\gamma_D(s | \pi^*)$ nur $U^\gamma_D(s)$, was dann der optimale erwartete Nutzen von $s$ ist. Das bedeutet, dass es keine Strategie gibt, die einen höheren erwarteten Nutzen für Zustand s liefert als π*. Dies ist ein zentraler Aspekt der Theorie des Verstärkungslernens und bildet die Grundlage für viele Algorithmen in diesem Bereich.

Sicher, hier ist die gegebene Information in Markdown und LaTeX formatiert:

**Beispiel 5.** Wir führen Beispiel 4 fort. Wir können nach den Überlegungen dort leicht sehen, dass

$$U^\gamma_D(s_{1,1}^1 | \pi_{vc}) = U^\gamma_D(\pi_{vc}) \approx 15.529$$

gilt. Schauen wir uns den Zustand $s_{0,0}^2$ an (der Agent ist in Raum $r2$ und beide Räume sind sauber). Jede von $\pi_{vc}$ in $s_{0,0}^2$ startende Episode $em$ (für $m \in \mathbb{N}$) hat die Form

$$em = (s_{0,0}^2, \text{{move}}, \underbrace{s_{0,0}^1, \text{{charge}}, s_t, \ldots}_{m \text{{-mal}}})$$

und damit die Wahrscheinlichkeit

$$P(em) = 0.9 \cdot 0.1^{m-1} \cdot 1$$

und den Nutzen

$$U^\gamma_D(em) = -\sum_{i=1}^{m} 0.9^{i-1}$$

Es folgt

$$U^\gamma_D(s_{0,0}^2 | \pi_{vc}) = -\sum_{m=1}^{\infty} \left(0.9 \cdot 0.1^{m-1} \sum_{i=1}^{m} 0.9^{i-1}\right) \approx -1.099$$

In diesem Beispiel betrachten wir den Zustand $s_{0,0}^2$, in dem der Agent sich im Raum $r2$ befindet und beide Räume sauber sind. Jede Episode $em$, die in $s_{0,0}^2$ startet und der Strategie $\pi_{vc}$ folgt, hat die Form:

$$em = (s_{0,0}^2, \text{{move}}, \underbrace{s_{0,0}^1, \text{{charge}}, s_t, \ldots}_{m \text{{-mal}}})$$

Die Wahrscheinlichkeit $P(em)$ für jede solche Episode ist gegeben durch:

$$P(em) = 0.9 \cdot 0.1^{m-1} \cdot 1$$

Der Nutzen $U^\gamma_D(em)$ für jede solche Episode ist gegeben durch:

$$U^\gamma_D(em) = -\sum_{i=1}^{m} 0.9^{i-1}$$

Daher ist der Nutzen $U^\gamma_D(s_{0,0}^2 | \pi_{vc})$ von $s_{0,0}^2$ unter der Strategie $\pi_{vc}$ gleich:

$$U^\gamma_D(s_{0,0}^2 | \pi_{vc}) = -\sum_{m=1}^{\infty} \left(0.9 \cdot 0.1^{m-1} \sum_{i=1}^{m} 0.9^{i-1}\right) \approx -1.099$$

Das bedeutet, dass der erwartete Nutzen des Zustands $s_{0,0}^2$ unter der Strategie $\pi_{vc}$ etwa $-1.099$ beträgt. Dieser negative Wert zeigt an, dass es für den Agenten nachteilig ist, sich in diesem Zustand zu befinden und der gegebenen Strategie zu folgen. Der Agent würde also versuchen, diesen Zustand zu vermeiden oder eine andere Strategie zu wählen, um den Nutzen zu maximieren.

## 4.1.3.Iterative Strategieentwicklung <a name="4_1_3"></a>


## Beipsielscode

In [5]:
class State:
    def __init__(self, robot_room, room1_clean, room2_clean):
        self.robot_room = robot_room
        self.room1_clean = room1_clean
        self.room2_clean = room2_clean

class Action:
    MOVE = "move"
    CLEAN = "clean"
    CHARGE = "charge"

class VacuumCleanerEnvironment:
    def __init__(self):
        self.states = [State(i, j, k) for i in range(1, 3) for j in [0, 1] for k in [0, 1]]
        self.actions = [Action.MOVE, Action.CLEAN, Action.CHARGE]
        self.state_transition_probabilities = self.initialize_state_transitions()
        self.rewards = self.initialize_rewards()

    def initialize_state_transitions(self):
        # Initialize state transition probabilities
        state_transitions = {}
        for state in self.states:
            for action in self.actions:
                state_transitions[(state, action)] = {}  # This will be a dictionary of {next_state: probability}
        return state_transitions

    def initialize_rewards(self):
        # Initialize rewards
        rewards = {}
        for state in self.states:
            for action in self.actions:
                rewards[(state, action)] = 0  # This will be the reward for performing the action in the given state
        return rewards

# Test program



All tests passed.


In [4]:
env = VacuumCleanerEnvironment()
print(env.states)
print(env.actions)


[<__main__.State object at 0x0000017C926C0590>, <__main__.State object at 0x0000017C91652290>, <__main__.State object at 0x0000017C926C0C90>, <__main__.State object at 0x0000017C926C15D0>, <__main__.State object at 0x0000017C926C3650>, <__main__.State object at 0x0000017C9164A750>, <__main__.State object at 0x0000017C926C89D0>, <__main__.State object at 0x0000017C926C8790>]
['move', 'clean', 'charge']


In [6]:
def test_vacuum_cleaner_environment():
    env = VacuumCleanerEnvironment()
    assert len(env.states) == 8, "There should be 8 states."
    assert len(env.actions) == 3, "There should be 3 actions."

    # Test state transitions
    for state in env.states:
        for action in env.actions:
            assert isinstance(env.state_transition_probabilities[(state, action)], dict), \
                "Each state-action pair should map to a dictionary of next states and probabilities."

    # Test rewards
    for state in env.states:
        for action in env.actions:
            assert isinstance(env.rewards[(state, action)], (int, float)), \
                "Each state-action pair should map to a numeric reward."

    print("All tests passed.")

if __name__ == "__main__":
    test_vacuum_cleaner_environment()

All tests passed.
